# Trabalho Final - Análise de Dados com Python

Integrantes:

    1) nome1
    2) nome2
    3) nome3
    4) nome4

### Enunciado

A partir dos dados housing.csv, realize toda análise exploratória necessária e prepare os dados para algoritmos de machine learning usando a Scikit-Learn. Treine um modelo de regressão linear que prediga a variável meta (median_house_value) e cujo valor de RMSE seja menor ou igual a 69000. 

In [219]:
#importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [220]:
# leitura dos dados
housing = pd.read_csv('bases/housing.csv')
#housing.head(3)

In [221]:
#housing.info()
#print()
#print('*'*60)
#print()
#housing.describe()

In [222]:
#housing.describe(include=['O'])

In [223]:
housing = housing[~housing['total_bedrooms'].isnull()]
x = housing['total_bedrooms']
Q1 =  x.quantile(.25)
Q3 =  x.quantile(.75)
IIQ = Q3 - Q1
limite_inferior = Q1 - 1.5 * IIQ
limite_superior = Q3 + 1.5 * IIQ
housing = housing[x.between(limite_inferior, limite_superior)]

In [224]:
#print("Antes", housing.shape)

housing = housing.drop(['longitude', 'latitude'], axis=1)

#print("Antes", housing.shape)

In [225]:
ocean_mapping = {"<1H OCEAN": 1, "INLAND": 2, "ISLAND": 3, "NEAR BAY": 4, "NEAR OCEAN": 5}
housing['ocean_proximity'] = housing['ocean_proximity'].map(ocean_mapping)
#housing['ocean_proximity'] = housing['ocean_proximity'].fillna(0)

#housing.head(3)

In [226]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.694686
total_rooms           0.199464
housing_median_age    0.111914
ocean_proximity       0.080895
households            0.074860
total_bedrooms        0.055051
population           -0.055965
Name: median_house_value, dtype: float64

In [227]:
features = housing.drop('median_house_value', axis=1)
labels = housing['median_house_value'].copy()

In [228]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [238]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

columns = features.columns.to_list()
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(columns)),
    ('std_scaler', StandardScaler())#,
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='median'))
])

In [239]:
features_prepared = num_pipeline.fit_transform(housing)
#features_prepared

In [240]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [241]:
X = features_prepared
y = labels.values
split_test_size = 0.20

In [242]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size)

In [243]:
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train.ravel()) # flatten arrays

LinearRegression()

In [244]:
y_pred = lr_model.predict(X_test)
#y_pred

In [245]:
import math
math.sqrt(mean_squared_error(y_test,y_pred))

77249.3279938783

In [246]:
lr_model.score(X_test,y_test)

0.5493630658008515